## V3
- Oneshot training without generator
- Normalise tempo

In [ ]:
!pip install pretty_midi
!pip install keras_self_attention
!pip install music21
!pip install mido

     |████████████████████████████████| 5.6 MB 5.1 MB/s 
     |████████████████████████████████| 51 kB 4.7 MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-py3-none-any.whl size=5591953 sha256=768b367d3b5b7a5d81eab1c581283a82c519dcf81b91de3d4cf1feb9e317f279
  Stored in directory: /root/.cache/pip/wheels/ad/74/7c/a06473ca8dcb63efb98c1e67667ce39d52100f837835ea18fa
Successfully built pretty-midi
  Created wheel for keras-self-attention: filename=keras_self_attention-0.50.0-py3-none-any.whl size=19414 sha256=16c8e5c13bec3b0ec3399e12e5f2dfc7732092438679eb5c2aee1daf60282ff0
  Stored in directory: /root/.cache/pip/wheels/92/7a/a3/231bef5803298e7ec1815215bc0613239cb1e9c03c57b13c14
Successfully built keras-self-attention


In [ ]:
import pretty_midi
from music21 import *
import pickle
import numpy as np
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import time
import os
import glob
from itertools import groupby
import mido
from mido import MidiFile, merge_tracks, tempo2bpm
import math

from keras.preprocessing import sequence
from keras.models import Sequential 
from keras.layers import Dense, LSTM, Bidirectional, Dropout, GlobalMaxPooling1D, Activation, GlobalMaxPooling2D
from keras_self_attention import SeqSelfAttention
#from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.layers import Layer
from tensorflow.python.client import device_lib
import keras

# data_path = "./selected_data/"
data_path = "./"
# data_path = "./sample_data/"
# encoded_data_path = "./encoded_doug_mckenzie_midi_32/"
encoded_data_path = "./encoded_classic_midi/"



## Parsing Midi file as vector

In [ ]:
def extract_midi_info(path):
    mid = converter.parse(path)
    max_idx = np.argmax(len(i) for i in mid)
    piano_part = mid[max_idx] #probably
    for i in piano_part:
        if isinstance(i, tempo.MetronomeMark):
            bpm = i.getQuarterBPM()
            break
    try:
        key = piano_part.keySignature
    except:
        print(f"Error while finding key signature for song {temp}")

    key_in_major = key.asKey(mode='major')
    offset_by = key_in_major.tonic.pitchClass
    return offset_by, bpm

def preprocess_midi(path, offset_by, bpm):
    mid = pretty_midi.PrettyMIDI(midi_file=path)
    filtered_inst_ls = [inst for inst in mid.instruments if ((len(inst.notes) > 0) and
                                                    (inst.is_drum == False) and
                                                    (inst.program < 8)
                                                   )]
    piano = filtered_inst_ls[np.argmax([len(inst.notes) for inst in filtered_inst_ls])]
            
    start_time = piano.notes[0].start
    end_time = piano.get_end_time()
    
    quater_note_len = 60/bpm
#     Set 4 for 16th note, 8 for 32 note
    nth_note = 8
#     Set fs to get 16th notes
    fs = 1/(quater_note_len/nth_note)
#     fs = 100
    
    piano_roll = piano.get_piano_roll(fs = fs, times = np.arange(start_time, end_time,1./fs))
    piano_roll = np.roll(piano_roll, -offset_by)
    out = np.where(piano_roll > 0, 1,0)
    
    return out.T

def process_piano_roll(piano_roll, max_consecutive = 64): 
#     This function is to remove consecutive notes that last for more than roughtly 2 secs
    prev = np.random.rand(128)
    count = 0
    remove_idxs = []
    remove_slice = []
    for idx, piano_slice in enumerate(piano_roll):
#         print(prev.shape)
#         print(piano_slice.shape)
        if(np.array_equal(prev, piano_slice)):
            count+=1
            if (count > max_consecutive):
                remove_idxs.append(idx)
                if (str(piano_slice) not in remove_slice):
                    remove_slice.append(str(piano_slice))
        else:
            count = 0
        prev = piano_slice
    out_piano_roll = np.delete(piano_roll, remove_idxs, axis=0)
    return out_piano_roll

failed_list = []
# keep track of list of midi we failed to parse and preprocess
for temp in glob.glob(data_path + "*.mid"):
    try:
        print(temp)
        offset_by, bpm = extract_midi_info(temp)
        piano_roll = preprocess_midi(temp, offset_by, bpm)
        piano_roll = process_piano_roll(piano_roll,128)
        name  = temp.split("/")[-1].split(".")[0]
        out_name = encoded_data_path + f'encoded_{name}.npy'
        print(out_name)
        print(piano_roll[10])
        np.save(out_name, piano_roll)
        print("1")
        print(f"saved {out_name}")
        
    except:
        print(f"Faield to preprocess {temp}")
        failed_list.append(temp)
        continue

./chpn_op10_e05.mid
./encoded_classic_midi/encoded_chpn_op10_e05.npy
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
1
saved ./encoded_classic_midi/encoded_chpn_op10_e05.npy
./clementi_opus36_1_2.mid
./encoded_classic_midi/encoded_clementi_opus36_1_2.npy
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
1
saved ./encoded_classic_midi/encoded_clementi_opus36_1_2.npy
./chpn-p24.mid
./encoded_classic_midi/encoded_chpn-p24.npy
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
!zip -r /content/encoded_classic_midi.zip /content/encoded_classic_midi

updating: content/encoded_classic_midi/ (stored 0%)
updating: content/encoded_classic_midi/encoded_liz_rhap15.npy (deflated 100%)
updating: content/encoded_classic_midi/encoded_grieg_zwerge.npy (deflated 100%)
updating: content/encoded_classic_midi/encoded_burg_gewitter.npy (deflated 100%)
updating: content/encoded_classic_midi/encoded_chpn-p12.npy (deflated 100%)
updating: content/encoded_classic_midi/encoded_haydn_35_2.npy (deflated 100%)
updating: content/encoded_classic_midi/encoded_clementi_opus36_4_2.npy (deflated 100%)
updating: content/encoded_classic_midi/encoded_mendel_op62_3.npy (deflated 100%)
updating: content/encoded_classic_midi/encoded_scn16_6.npy (deflated 100%)
updating: content/encoded_classic_midi/encoded_chpn-p22.npy (deflated 100%)
updating: content/encoded_classic_midi/encoded_mendel_op62_5.npy (deflated 100%)
updating: content/encoded_classic_midi/encoded_schub_d960_1.npy (deflated 100%)
updating: content/encoded_classic_midi/encoded_haydn_8_1.npy (deflated 100%

In [ ]:
from google.colab import files
files.download("/content/encoded_classic_midi.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>